###  Created by Luis A. Sanchez-Perez (alejand@umich.edu).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

**NOTE:** Do not modify any of the cells (unless specified otherwise) provided in this notebook or you might have problems finishing the assignment.

<a id='part-i'></a>
## Part I - Overview

In this assignment you will implement, visualize and test different optimizers that represent a huge improvement over a basic gradient descend optimizer and do not need to compute/estimate the Hessian matrix. The optimizers you will compare are:
* Basic gradient descend (Already implemented as example: `BatchGradientDescendOptimizer`)
* Adagrad (Already implemented as example: `AdagradOptimizer`)
* Gradient descent with Momentum
* RMSProp
* Adam

There are more optimizers you might find out there (included upgraded versions) but in general, learning the ones abovementioned will give the tools to understand other related optimizers.

We will minimize (as cost functions) some of the functions (with two independant variables `x` and `y`) from a [well known set of functions](https://en.wikipedia.org/wiki/Test_functions_for_optimization) that are useful to evaluate optimization algorithms. These functions are:
* The Beale function
* The Booth function
* The Himmelblau function
* The Goldstein-Price function

All these functions are implemented in the library `cost_functions.py` that must be downloaded together with this notebook. The gradient of these functions is computed using the [autograd library](https://github.com/HIPS/autograd).

You don't have to worry about implementing the training loop (implemented in the `optimizers_starter.py` library) or the visualization (implemented in the `training_animation.py` library). Your only task will be to correctly implement each of the optimizers following the optimizers template.

This notebook consists of the following parts:
* <p><a href="#part-i")>Part I - Overview:</a> A brief description of the assignment.</p>
* <p><a href="#part-ii")>Part II - Implementing the optimizers:</a> You will code only in this part.</p>
* <p><a href="#part-iii")>Part III - Training implementation:</a> Using the optimizers to minimize the functions.</p>
* <p><a href="#part-iv")>Part IV - Testing and visualizing trainings:</a> A nice visualization of the trainings for different cost functions and optimizers.</p>

<p><span style="color:red"><b>Note: </b> I recommend you run the entire notebook before coding anything, paying particular attention to all cells outputs in <a href="#part-iv")>Part IV</a></span></p>

### Installing autograd
If you don't have the autograd library already installed you must run the cell below in order to properly finish this assigment.

In [ ]:
# Do not modify this cell!
!pip install autograd

### Libraries
All the libraries specified below are needed to finish this assigment. You must download and place in the same folder this notebook and the following files:
* `cost_functions.py`
* `training_animation.py`
* `optimizers_starter.py`
* `optimizers_unit_test.py`

In [ ]:
# Do not modify this cell!
import numpy as np
from cost_functions import graph_function
from cost_functions import beale, gradient_beale
from cost_functions import booth, gradient_booth
from cost_functions import himmelblau, gradient_himmelblau
from cost_functions import goldstein, gradient_goldstein
from cost_functions import approximate_gradient
from training_animation import TrainingAnimation
from optimizers_unit_tests import *
try:
    optimizers = __import__('optimizers_solution')
except ImportError:
    optimizers = __import__('optimizers_starter')
BatchGradientDescendOptimizer = optimizers.BatchGradientDescendOptimizer
AdagradOptimizer = optimizers.AdagradOptimizer
MomentumOptimizer = optimizers.MomentumOptimizer
RMSPropOptimizer = optimizers.RMSPropOptimizer
AdamOptimizer = optimizers.AdamOptimizer
train = optimizers.train

### A simple but beatiful test!
In order to verify that `autograd` and our functions from `cost_functions.py` are working properly we are going to compute the gradient of the `himmelblau` function in three different ways (including `autograd`) at the point (1,2).

In [ ]:
# Do not modify this cell!
x = 1.
y = 2.
value = gradient_himmelblau([x, y])
print('Gradient using autograd: ', value)
value = [
    4*x*(x**2 + y - 11) + 2*(x + y**2 - 7),
    2*(x**2 + y - 11) + 4*y*(x + y**2 - 7)
]
print('Gradient analytically computed: ', value)
delta = 1e-5
value = [
    (himmelblau([x + delta, y]) - himmelblau([x - delta, y])) / (2*delta),
    (himmelblau([x, y + delta]) - himmelblau([x, y - delta])) / (2*delta)
]        
print('Gradient using finite differences: ', value)

<a id='part-ii'></a>
## Part II - Implementing the optimizers
In the file `optimizers_starter.py` you will find definitions for the following classes:
* `Param`: A class to hold parameters.
* `Optimizer`: An abstract class (base class) for any optimizer implemented later.
* `BatchGradientDescendOptimizer`: An optimizer class implementing the update rule used in Batch Gradient Descend.
* `AdagradOptimizer`: An optimizer class updating parameters using Adagrad (adaptative learning rate).

Each `Optimizer` has basically three methods:
* `__init__ (self, **opts)`: Method called when the object is created and receives a list of keywords parameters in `**opts`.
* `update(self, params, gradients)`: Method called to update parameters passed in the iterable `params` based on their corresponding `gradients`. Notice `len(params) == len(gradients)`, so there is an one-to-one correspondance between their entries.
* `reset(self, params)`: Method called to reset any optimizer configuration for the parameters passed in the iterable `params`. This function is called before starting training for all trainable parameters.

You will have to complete the following implementatons: `MomentumOptimizer`, `RMSPropOptimizer` and `AdamOptimizer`. In order to do so you must complete methods: `__init__`, `reset` and `update`.

<p><span style="color:red"><b>Note: </b> Review the implementations for <b>BatchGradientDescendOptimizer</b> and <b>AdagradOptimizer</b> to better understand how to use these methods</span></p>

### Q1. Implement Gradient Descend with Momentum optimizer
In this part you are asked to implement Gradient Descend with Momentum. Your optimizer must have at least the following attributes (which are already setup):
* `alpha`: Learning rate.
* `beta`: Parameter to control the exponetially weighted average of past gradients (momentum).
* `bias`: When true you optimizer must use bias correction in the computation of the momentum. correction.

**Indications**
* You are required to complete the `reset` and `update` methods (here is where the parameters should be updated).
* You can add anything to `__init__` and create new methods if needed.
* Your implementaton must also include bias correction.

<p><span style="color:red"><b>Note: </b> Go to <b>optimizers_starter.py</b> to complete your implementation. In there you should find a partial implementation.</span></p>

### Unit tests
A good indication that your implementation is going well is not getting any `Failed` messages after running the cell below.

In [ ]:
# Do not modify this cell!
print('Verifying update method (without bias correction): ', end='')
print('Passed') if momentum_update_default_unit_test() else print('Failed')
print("Verifying update method (with bias correction): ", end='')
print('Passed') if momentum_update_with_bias_unit_test() else print('Failed')
print("Verifying reset method: ", end='')
print('Passed') if momentum_reset_unit_test() else print('Failed')

### Q2. Implement RMSProp optimizer
In this part you are asked to implement RMSProp. Your optimizer must have at least the following attributes (which are already setup):
* `alpha`: Learning rate.
* `beta`: Parameter to control the exponetially weighted average of past gradients (momentum).
* `epsilon`: Small positive value to avoid getting zero in the denominator.
* `bias`: When true you optimizer must use bias correction in the computation of the momentum.
* `decay`: Indicates the learning rate decay. When zero, no decay is used.

**Indications**
* You are required to complete the `reset` and `update` methods (here is where the parameters should be updated).
* You can add anything to `__init__` and create new methods if needed.
* Your implementaton must also include bias correction.
* Your implementaton must also include learning rate decay using the following formula:

$$ \alpha^{'} = (\frac{1}{1 + \text{decay} * \text{epoch}}) * \alpha$$

where:
* $\alpha^{'}$: is the actual learning rate you should to update the parameter in question.
* $\alpha$ is the value passed as argument `alpha` to the optimizer.
* $\text{decay}$: is the value passed as argument `decay` to the optimizer.
* $\text{epoch}$: is the epoch number, which you could get by tracking how many times the optimizer update method has been called for the parameter in question.

<p><span style="color:red"><b>Note: </b> Go to <b>optimizers_starter.py</b> to complete your implementation. In there you should find a partial implementation.</span></p>

### Unit tests
A good indication that your implementation is going well is not getting any `Failed` messages after running the cell below.

In [ ]:
# Do not modify this cell!
print("Verifying update method (without bias correction or decay): ", end='')
print('Passed') if rmsprop_update_default_unit_test() else print('Failed')
print('Verifying update method (with bias correction): ', end='')
print('Passed') if rmsprop_update_with_bias_unit_test() else print('Failed')
print('Verifying update method (with bias correction and decay): ', end='')
print('Passed') if rmsprop_update_with_bias_decay_unit_test() else print('Failed')
print("Verifying reset method: ", end='')
print('Passed') if rmsprop_reset_unit_test() else print('Failed')

### Q3. Implement Adam optimizer
In this part you are asked to implement the Adam optimizer. Your optimizer must have at least the following attributes (which are already setup):
* `alpha`: Learning rate.
* `beta1`: Parameter to control the exponetially weighted average of past gradients (momentum).
* `beta2`: Parameter to control the exponetially weighted average of past gradients squared (to normalize as Adagrad).
* `epsilon`: Small positive value to avoid getting zero in the denominator.
* `bias`: When true you optimizer must use bias correction in the computation of the momentum.
* `decay`: Indicates the learning rate decay. When zero, no decay is used.


**Indications**
* You are required to complete the `reset` and `update` methods (here is where the parameters should be updated).
* You can add anything to `__init__` and create new methods if needed.
* Your implementaton must also include bias correction.
* Your implementaton must also include learning rate decay using the following formula:

$$ \alpha^{'} = (\frac{1}{1 + \text{decay} * \text{epoch}}) * \alpha$$

where:
* $\alpha^{'}$: is the actual learning rate you should to update the parameter in question.
* $\alpha$ is the value passed as argument `alpha` to the optimizer.
* $\text{decay}$: is the value passed as argument `decay` to the optimizer.
* $\text{epoch}$: is the epoch number, which you could get by tracking how many times the optimizer update method has been called for the parameter in question.

<p><span style="color:red"><b>Note: </b> Go to <b>optimizers_starter.py</b> to complete your implementation. In there you should find a partial implementation.</span></p>

### Unit tests
A good indication that your implementation is going well is not getting any `Failed` messages after running the cell below.

In [ ]:
# Do not modify this cell!
print("Verifying update method (withou| bias correction or decay): ", end='')
print('Passed') if adam_update_default_unit_test() else print('Failed')
print('Verifying update method (with bias correction): ', end='')
print('Passed') if adam_update_with_bias_unit_test() else print('Failed')
print('Verifying update method (with bias correction and decay): ', end='')
print('Passed') if adam_update_with_bias_decay_unit_test() else print('Failed')
print("Verifying reset method: ", end='')
print('Passed') if adam_reset_unit_test() else print('Failed')

<a id='part-iii'></a>
## Part III - Training implementation

The implementation of the training loop is given in the function `train` imported from the file `optimizers_starter.py`. Although you don't have to add or change anything in this code is good to review and see how the optimizers definitions are used. Notice:
* A training is performed for each optimizer passed in `optimizers`.
* The function to minimize is defined by the callable `cost_func` and its gradient is computed using `grad_func`.
* Results for each optimizer are stored in a dictionary named `results` where each optimizer is a key.

<a id='part-iv'></a>
## Part IV - Testing and visualizing trainings 
In the cells following below we will test some optimizers configurations training four different cost functions.

### Beale cost function

#### Configuring optimizers and training

In [ ]:
# Starting point to initialize the params
point = [-2,3]
# Creates different optimizers to train on
optimizers = [
    BatchGradientDescendOptimizer(alpha=1e-4),
    AdagradOptimizer(alpha=1),
    MomentumOptimizer(alpha=1e-4, bias=True),
    RMSPropOptimizer(alpha=1, decay=0.1),
    AdamOptimizer(alpha=1)
]
# Trains with the different optimizers
log = train(point, optimizers, beale, gradient_beale)

#### Plotting cost function and its gradient

In [ ]:
graph_function(beale, xrange=[-4.5, 4.5], yrange=[-4.5, 4.5], title='Beale')

#### Showing cool animation of the training!
You can press `Esc` anytime to finish animation!

In [ ]:
TrainingAnimation(log, beale, gradient_beale).start()

### Booth cost function

#### Configuring optimizers and training

In [ ]:
# Starting point to initialize the params
point = [0,-10]
# Creates different optimizers to train on
optimizers = [
    BatchGradientDescendOptimizer(alpha=1e-2),
    AdagradOptimizer(alpha=1),
    MomentumOptimizer(alpha=1e-2, bias=True),
    RMSPropOptimizer(alpha=1, decay=0.1),
    AdamOptimizer(alpha=1)
]
log = train(point, optimizers, booth, gradient_booth)

#### Plotting cost function and its gradient

In [ ]:
graph_function(booth, xrange=[-10, 10], yrange=[-10, 10], title='Booth')

#### Showing cool animation of the training!
You can press `Esc` anytime to finish animation!

In [ ]:
TrainingAnimation(log, booth, gradient_booth).start()

### Himmelblau cost function

#### Configuring optimizers and training

In [ ]:
# Starting point to initialize the params
point = [0,4]
# Creates different optimizers to train on
optimizers = [
    BatchGradientDescendOptimizer(alpha=1e-3),
    AdagradOptimizer(alpha=1e-1),
    MomentumOptimizer(alpha=1e-3, bias=True),
    RMSPropOptimizer(alpha=1e-1, decay=0.1),
    AdamOptimizer(alpha=1e-1)
]
log = train(point, optimizers, himmelblau, gradient_himmelblau)

#### Plotting cost function and its gradient

In [ ]:
graph_function(himmelblau, xrange=[-5, 5], yrange=[-5, 5], title='Himmelblau')

#### Showing cool animation of the training!
You can press `Esc` anytime to finish animation!

In [ ]:
TrainingAnimation(log, himmelblau, gradient_himmelblau).start()

### Goldstein cost function

#### Configuring optimizers and training

In [ ]:
# Starting point to initialize the params
point = [0,0]
# Creates different optimizers to train on
optimizers = [
    BatchGradientDescendOptimizer(alpha=1e-5),
    AdagradOptimizer(alpha=1e-1),
    MomentumOptimizer(alpha=1e-5, bias=True),
    RMSPropOptimizer(alpha=1e-1, decay=0.1),
    AdamOptimizer(alpha=1e-1)
]
log = train(point, optimizers, goldstein, gradient_goldstein)

#### Plotting cost function and its gradient

In [ ]:
graph_function(goldstein, xrange=[-2, 2], yrange=[-2, 2], title='Goldstein')

#### Showing cool animation of the training!
You can press `Esc` anytime to finish animation!

In [ ]:
TrainingAnimation(log, goldstein, gradient_goldstein).start()